In [1]:
import sys
!{sys.executable} -m pip install pycaret

In [2]:
import pandas as pd
df = pd.read_csv("../data/creditcard.csv")

In [3]:
data_unseen = df.sample(frac=0.1, random_state=42)      # Sample 10% of the data to become the unseen test set
df = df.drop(data_unseen.index)                       # Use the remaining 90% as the training (& validation) data

df.reset_index(drop=True, inplace=True)
data_unseen.reset_index(drop=True, inplace=True)

print('Data for Model Training & Validation: ' + str(df.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

Data for Model Training & Validation: (256326, 31)
Unseen Data For Predictions: (28481, 31)


In [4]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
1,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
2,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
3,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
4,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0


In [6]:
from pycaret.classification import *

In [7]:
expt_basic = setup(
    data=df, 
    session_id=42,                      # Random seed to ensure reproducibility of the experiment with the same data
    train_size=0.8,                     # 80% training data & 20% held-out validation data
    target='Class',
    feature_selection=True,
    fix_imbalance=True,
    numeric_imputation="median",        # "mean" by default
    categorical_imputation="mode",      # "constant" (not_available) by default
)

,Description,Value
0,session_id,42
1,Target,Class
2,Target Type,Binary
3,Label Encoded,"0: 0, 1: 1"
4,Original Data,"(256326, 31)"
5,Missing Values,False
6,Numeric Features,30
7,Categorical Features,0
8,Ordinal Features,False
9,High Cardinality Features,False


In [8]:
models()

,Name,Reference,Turbo
ID,,,
lr,Logistic Regression,sklearn.linear_model._logistic.LogisticRegression,True
knn,K Neighbors Classifier,sklearn.neighbors._classification.KNeighborsCl...,True
nb,Naive Bayes,sklearn.naive_bayes.GaussianNB,True
dt,Decision Tree Classifier,sklearn.tree._classes.DecisionTreeClassifier,True
svm,SVM - Linear Kernel,sklearn.linear_model._stochastic_gradient.SGDC...,True
rbfsvm,SVM - Radial Kernel,sklearn.svm._classes.SVC,False
gpc,Gaussian Process Classifier,sklearn.gaussian_process._gpc.GaussianProcessC...,False
mlp,MLP Classifier,sklearn.neural_network._multilayer_perceptron....,False
ridge,Ridge Classifier,sklearn.linear_model._ridge.RidgeClassifier,True


In [ ]:
best5 = compare_models(sort="F1", exclude=["rbfsvm","gpc","mlp"], n_select=5)

IntProgress(value=0, description='Processing: ', max=54)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,23:11:43
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9995,0.9637,0.8128,0.8652,0.8368,0.8365,0.8376,85.3360
dt,Decision Tree Classifier,0.9976,0.8855,0.7731,0.3988,0.5240,0.5229,0.5530,10.4700
nb,Naive Bayes,0.9922,0.9639,0.7590,0.1513,0.2521,0.2500,0.3365,0.3700
ridge,Ridge Classifier,0.9887,0.0000,0.8246,0.1147,0.2013,0.1989,0.3049,0.3890
ada,Ada Boost Classifier,0.9869,0.9734,0.8695,0.1050,0.1871,0.1846,0.2991,35.8100
svm,SVM - Linear Kernel,0.7614,0.0000,0.7551,0.1063,0.1610,0.1585,0.2113,13.3910
lr,Logistic Regression,0.9780,0.9643,0.8723,0.0700,0.1288,0.1260,0.2412,3.6950
qda,Quadratic Discriminant Analysis,0.9743,0.9615,0.8636,0.0552,0.1038,0.1009,0.2146,1.4150
knn,K Neighbors Classifier,0.9403,0.7561,0.5267,0.0151,0.0294,0.0262,0.0817,3.6750


In [ ]:
for model in best5:
    print(model)
    print()